### GARCH(2, 2) model
$$a_t = \varepsilon_t\sqrt{\omega + a_1a_{t-1}^2+a_2a_{t-2}^2 + \beta_1\sigma_{t-1}^2+\beta_2\sigma_{t-2}^2} $$
$$a_0, a_1 \approx 𝒩(0, 1) $$
$$\sigma_0 = 1, \sigma_1 = 1$$
$$\varepsilon_t \approx 𝒩 (0, 1)$$

In [ ]:
pip install arch

In [ ]:
from random import gauss
import matplotlib.pyplot as plt
import numpy as np
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

n = 1000
omega = 0.5

alpha_1 = 0.1
alpha_2 = 0.2

beta_1 = 0.3
beta_2 = 0.4

test_size = int(n*0.1)

series = [gauss(0, 1), gauss(0.1)]
vols = [1] * 2

for _ in range(n):
  new_vol = np.sqrt(omega+alpha_1*series[-1]**2 + alpha_2*series[-2]**2 + beta_1*vols[-1]**2 + beta_2*vols[-2]**2)
  new_val = gauss(0, 1) * new_vol

  vols.append(new_vol)
  series.append(new_val)

plt.plot(series)
plt.plot(vols, color="red")
plt.title("Volatility and Data")

In [ ]:
train, test = series[:-test_size], series[-test_size:]
model = arch_model(train, p=2, q=2)
model_fit = model.fit()

print(model_fit.summary())

pred = model_fit.forecast(horizon=test_size)
plt.plot(vols[-test_size:])
plt.plot(np.sqrt(pred.variance.values[-1, :]), color="orange")
plt.legend(["actual", "predicted"])

In [ ]:
rolling_preds = []
for i in range(test_size):
  train = series[:-(test_size-i)]
  model = arch_model(train, p=2, q=2)
  model_fit= model.fit(disp="off")
  pred = model_fit.forecast(horizon = 1)
  rolling_preds.append(np.sqrt(pred.variance.values[-1][0]))

plt.plot(vols[-test_size:])
plt.plot(rolling_preds, color="orange")
plt.legend(["actual", "predicted"])
plt.title("vol prediction - rolling")